In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
std = pd.read_csv("../Data/FE/crime_num_fe.csv")
pca = pd.read_csv("../Data/FE/crime_num_dr.csv")
pca = pca.iloc[:, 1:]
vs = pd.read_csv("../Data/FE/crime_num_varselect.csv")
vs = vs.iloc[:, 1:]

y = std['Num_Crimes']

# Train Test Split (just want idx)

In [3]:
X = std.drop(['Num_Crimes'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=308, test_size=.2)
train_idx = X_train.index
test_idx = X_test.index

STD

In [20]:
X = std.drop(['Num_Crimes', 'Date','Day', 'Month', 'Year','TS_interval'], axis=1)
X_train = X.iloc[train_idx,]
X_test = X.iloc[test_idx,]

In [21]:
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [22]:
Xt_train = preprocessor.fit_transform(X_train)
lcv = LassoCV(cv=5, alphas=np.logspace(-6, 6, 10)).fit(Xt_train, y_train)

/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 53373.963664172596, tolerance: 47.811655030614574
  model = cd_fast.enet_coordinate_descent_gram(
/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 140376.18983551915, tolerance: 47.811655030614574
  model = cd_fast.enet_coordinate_descent_gram(
/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 151689.46604433458, tolerance: 47.811655030614574
  model = cd_fast.enet_coordinate_descent_gram(
/Users/jacksonpassey/anaconda3/lib/py

In [23]:
# validation
Xt_test = preprocessor.transform(X_test)
preds = lcv.predict(Xt_test)
print(f"RMSE: {mean_squared_error(y_test, preds, squared=False)}") # rmse
print(f"Var Red: {np.corrcoef(y_test, preds)[0,1]**2}") # var reduction

RMSE: 4.718288242680718
Var Red: 0.3363159256745683


In [24]:
# on original data
Xt = preprocessor.transform(X)
preds = lcv.predict(Xt)
print(f"RMSE: {mean_squared_error(y, preds, squared=False)}") # rmse
print(f"Var Red: {np.corrcoef(y, preds)[0,1]**2}") # var reduction

RMSE: 4.81325644632486
Var Red: 0.3531580677732623


In [25]:
np.savetxt("PyPreds/lassopreds.csv", preds, delimiter=",")

PCA

In [9]:
X = pca.drop(['Num_Crimes', 'Date','Day', 'Month', 'Year','TS_interval'], axis=1)
X_train = X.iloc[train_idx,]
X_test = X.iloc[test_idx,]

In [10]:
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [11]:
Xt_train = preprocessor.fit_transform(X_train)
lcv = LassoCV(cv=5, alphas=np.logspace(-6, 6, 10)).fit(Xt_train, y_train)

/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 331.2772948954953, tolerance: 47.811655030614574
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 386.8744106934755, tolerance: 47.811655030614574
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1024.9357768218033, tolerance: 47.811655030614574
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/jacksonpassey/anaconda3/li

In [12]:
# validation
Xt_test = preprocessor.transform(X_test)
preds = lcv.predict(Xt_test)
print(f"RMSE: {mean_squared_error(y_test, preds, squared=False)}") # rmse
print(f"Var Red: {np.corrcoef(y_test, preds)[0,1]**2}") # var reduction

RMSE: 4.774459841732658
Var Red: 0.32037796752442593


In [13]:
Xt = preprocessor.transform(X)
preds = lcv.predict(Xt)
print(f"RMSE: {mean_squared_error(y, preds, squared=False)}") # rmse
print(f"Var Red: {np.corrcoef(y, preds)[0,1]**2}") # var reduction

RMSE: 4.868930606280533
Var Red: 0.3381061818621572


VS

In [14]:
X = vs.drop(['Num_Crimes', 'Date','Day', 'Month', 'Year','TS_interval'], axis=1)
X_train = X.iloc[train_idx,]
X_test = X.iloc[test_idx,]

In [15]:
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [16]:
Xt_train = preprocessor.fit_transform(X_train)
lcv = LassoCV(cv=5, alphas=np.logspace(-6, 6, 10)).fit(Xt_train, y_train)

/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 128807.9715680755, tolerance: 47.811655030614574
  model = cd_fast.enet_coordinate_descent_gram(
/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 153343.4423501815, tolerance: 47.811655030614574
  model = cd_fast.enet_coordinate_descent_gram(
/Users/jacksonpassey/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 140997.99058247884, tolerance: 47.99970384760753
  model = cd_fast.enet_coordinate_descent_gram(
/Users/jacksonpassey/anaconda3/lib/pytho

In [17]:
# validation
Xt_test = preprocessor.transform(X_test)
preds = lcv.predict(Xt_test)
print(f"RMSE: {mean_squared_error(y_test, preds, squared=False)}") # rmse
print(f"Var Red: {np.corrcoef(y_test, preds)[0,1]**2}") # varuabce reduction

RMSE: 4.749139092011689
Var Red: 0.32782558091670244


In [18]:
Xt = preprocessor.transform(X)
preds = lcv.predict(Xt)
print(f"RMSE: {mean_squared_error(y, preds, squared=False)}") # rmse
print(f"Var Red: {np.corrcoef(y, preds)[0,1]**2}") # varuabce reduction

RMSE: 4.849853915604818
Var Red: 0.3433015630667279


In [19]:
np.logspace(-6, 6, 10)

array([1.00000000e-06, 2.15443469e-05, 4.64158883e-04, 1.00000000e-02,
       2.15443469e-01, 4.64158883e+00, 1.00000000e+02, 2.15443469e+03,
       4.64158883e+04, 1.00000000e+06])